In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
%matplotlib inline

import os, pickle, re, glob, time
import pandas as pd
import numpy as np
np.set_printoptions(precision=2)

import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
from scipy import stats
from collections import Counter

sns.set_style('ticks')
pd.set_option('precision', 2)
#np.set_printoptions('precision', 2)

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from lib import utils

In [2]:
args = pickle.load(open('arguments.pickle', 'rb'))

# Load reference

In [4]:
seq_ref_list = pickle.load(open(os.path.join(args.preprocess_dir, 'seq_ref_list.pickle'), 'rb'))

# สร้าง array สำหรับ insertion

In [6]:
insertion_df = pd.read_csv(os.path.join(args.preprocess_dir, 'insertion_df.csv'))
startstop_df = pd.read_csv(os.path.join(args.preprocess_dir, 'startstop_df.csv'), index_col='id')

#เรียง id
id_list = sorted(list(set(startstop_df.index)))
len(id_list)

3406

In [7]:
insertion_df

,id,position_ref,n_insert_base,start_end,start_new,end_new
0,1081930,28250.0,1.0,10_29801,10,29801
1,1081930,28251.0,1.0,10_29801,10,29801
2,1315627,28250.0,1.0,63_29676,63,29676
3,1315627,28251.0,1.0,63_29676,63,29676
4,1315628,28250.0,3.0,28123_29766,28123,29766
5,1519355,28262.0,4.0,89_29740,89,29740
6,1543939,28262.0,4.0,45_29792,45,29792
7,1620144,28262.0,4.0,50_29759,50,29759
8,1914659,28262.0,4.0,18_29787,18,29787
9,1914660,28262.0,4.0,22260_29793,22260,29793


In [8]:
max_insertion = insertion_df.groupby(['position_ref']).max()

max_insertion = max_insertion.reset_index()
max_insertion.head()

,position_ref,id,n_insert_base,start_end,start_new,end_new
0,28250.0,1315628,3.0,63_29676,28123,29801
1,28251.0,1315627,1.0,63_29676,63,29801
2,28262.0,2621963,4.0,89_29740,22260,29891


In [9]:
max_insertion['sum'] = np.cumsum(max_insertion['n_insert_base'])
max_insertion = max_insertion[['position_ref','sum']]
max_insertion.head()

,position_ref,sum
0,28250.0,3.0
1,28251.0,4.0
2,28262.0,8.0


In [10]:
n_insertion = list(max_insertion['sum'])[-1]

In [11]:
#สร้าง array ขนาด sample x insertion จากนั้นใส่่ 1 ในตำแหน่่งที่มี
insertion_array = np.zeros((len(id_list),int(n_insertion)), dtype=str)

In [12]:
insertion_array.shape

(3406, 8)

In [13]:
s = 0
for p in list(max_insertion['position_ref']):
    temp_df = insertion_df[insertion_df['position_ref']==p]
    list_temp = list(temp_df['id'])

    for sample in list_temp:
        start = s
#             print(start)
        num = int(temp_df[temp_df['id']==sample]['n_insert_base'])
        i = id_list.index(sample)
        print(i)
        insertion_array[i][start:start+num] = 1
        print(p,insertion_array[i])
    s = int(max_insertion[max_insertion['position_ref']==p]['sum'].values)
    print(s)

1831
28250.0 ['1' '' '' '' '' '' '' '']
1898
28250.0 ['1' '' '' '' '' '' '' '']
1899
28250.0 ['1' '1' '1' '' '' '' '' '']
3
1831
28251.0 ['1' '' '' '1' '' '' '' '']
1898
28251.0 ['1' '' '' '1' '' '' '' '']
4
2003
28262.0 ['' '' '' '' '1' '1' '1' '1']
2068
28262.0 ['' '' '' '' '1' '1' '1' '1']
2128
28262.0 ['' '' '' '' '1' '1' '1' '1']
2401
28262.0 ['' '' '' '' '1' '1' '1' '1']
2402
28262.0 ['' '' '' '' '1' '1' '1' '1']
2406
28262.0 ['' '' '' '' '1' '1' '1' '1']
3160
28262.0 ['' '' '' '' '1' '1' '1' '1']
3161
28262.0 ['' '' '' '' '1' '1' '1' '1']
8


In [14]:
insertion_array[insertion_array==''] = '0'

In [15]:
insertion_array

array([['0', '0', '0', ..., '0', '0', '0'],
       ['0', '0', '0', ..., '0', '0', '0'],
       ['0', '0', '0', ..., '0', '0', '0'],
       ...,
       ['0', '0', '0', ..., '0', '0', '0'],
       ['0', '0', '0', ..., '0', '0', '0'],
       ['0', '0', '0', ..., '0', '0', '0']], dtype='<U1')

In [16]:
pickle.dump(insertion_array, open(os.path.join(args.preprocess_dir, 'insertion_array.pickle'), 'wb'))